In [75]:
import tabula
import pandas as pd
import pycountry
from datetime import datetime

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
all_tables = tabula.read_pdf("https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200321-sitrep-61-covid-19.pdf", pages='all', pandas_options={'header': None})


In [93]:
country_data = pd.DataFrame([],columns=[])

# Remove all tables without 7 columns
for df in all_tables:
    if len(df.columns) == 7:
        #country_data = country_data.append(df, ignore_index=True, sort=True)
        country_data = pd.concat([country_data,df],ignore_index=True, sort=True)

# Remove columns with null country or cases 
country_data = country_data[country_data[0].notnull()]
country_data = country_data[country_data[1].notnull()]
# header row
country_data = country_data[country_data[6]!='reported case']

# Rename columns
country_data = country_data.rename(columns={0:'Country',1:'Total_Cases',2:'Cases_New',3:'Deaths',4:'Deaths_New',5:'Transmission_Classification',6:'Days_Since_Last_Reported_Case'})

#country_data['Country/Name'] = pd.eval( pycountry.countries.search_fuzzy( country_data.0 ))

country_data.to_csv("20200321-sitrep-61-covid-19.csv",index=False)

## Adding timestamp

Before we save the file locally, we add the `Last_Update_Date` in `UTC` time zone.

In [95]:
country_data["Last_Update_Date"] = datetime.utcnow()


NameError: name 'datetime' is not defined

In [ ]:
country_data.to_csv(output_folder + "WHO_SITUATION_REPORTS.csv", index=False)